In [302]:
import pandas as pd
import numpy as np
import keras
from keras.layers import Input, Dense, LSTM, Embedding, concatenate
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [303]:
df= pd.read_csv("income.csv")

In [304]:
df

age         workclass  fnlwgt     education  education.num  \
0       40  Self-emp-not-inc  223881   Prof-school             15   
1       30           Private  149118       HS-grad              9   
2       46           Private  109209  Some-college             10   
3       32           Private  229566     Assoc-voc             11   
4       54                 ?  148657     Preschool              1   
...    ...               ...     ...           ...            ...   
24995   40           Private  130834  Some-college             10   
24996   31         Local-gov   33124     Bachelors             13   
24997   38       Federal-gov  190895     Bachelors             13   
24998   23           Private  420973     Bachelors             13   
24999   60           Private   88055          10th              6   

           marital.status        occupation   relationship   race     sex  \
0      Married-civ-spouse    Prof-specialty        Husband  White    Male   
1                Divorced      Craft-repair  Not-in-family  White  Female   
2      Married-civ-spouse      Adm-clerical        Husband  White    Male   
3      Married-civ-spouse     Other-service        Husband  White    Male   
4      Married-civ-spouse                 ?           Wife  White  Female   
...                   ...               ...            ...    ...     ...   
24995       Never-married      Adm-clerical  Not-in-family  White  Female   
24996       Never-married    Prof-specialty  Not-in-family  White  Female   
24997  Married-civ-spouse    Prof-specialty        Husband  White    Male   
24998       Never-married    Prof-specialty  Not-in-family  White  Female   
24999  Married-civ-spouse  Transport-moving        Husband  White    Male   

       capital.gain  capital.loss  hours.per.week native.country income  
0             99999             0              70  United-States   >50K  
1                 0             0              40  United-States  <=50K  
2                 0             0              40  United-States   >50K  
3                 0             0              60  United-States   >50K  
4                 0             0              40         Mexico  <=50K  
...             ...           ...             ...            ...    ...  
24995             0             0              40  United-States  <=50K  
24996             0             0              50  United-States  <=50K  
24997             0             0              40              ?   >50K  
24998             0             0              40  United-States  <=50K  
24999             0             0              40  United-States  <=50K  

[25000 rows x 15 columns]

In [305]:
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

In [306]:
#ako imaavme missing values za kategoriski
# categorical_columns = ['education','workclass']
# imputer = SimpleImputer(strategy = 'most_frequent')
# df[categorical_columns] = imputer.fit_transform(df[categorical_columns])

In [307]:
#ako imaavme missing values za numericki  i kategoriski


# numerical_culumns = ['hours.per.week']
# imputer_knn = KNNImputer(n_neighbors=5)
# df[numerical_culumns] = imputer_knn.fit_transform(df[numerical_culumns])

In [308]:
df

age         workclass  fnlwgt     education  education.num  \
0       40  Self-emp-not-inc  223881   Prof-school             15   
1       30           Private  149118       HS-grad              9   
2       46           Private  109209  Some-college             10   
3       32           Private  229566     Assoc-voc             11   
4       54                 ?  148657     Preschool              1   
...    ...               ...     ...           ...            ...   
24995   40           Private  130834  Some-college             10   
24996   31         Local-gov   33124     Bachelors             13   
24997   38       Federal-gov  190895     Bachelors             13   
24998   23           Private  420973     Bachelors             13   
24999   60           Private   88055          10th              6   

           marital.status        occupation   relationship   race     sex  \
0      Married-civ-spouse    Prof-specialty        Husband  White    Male   
1                Divorced      Craft-repair  Not-in-family  White  Female   
2      Married-civ-spouse      Adm-clerical        Husband  White    Male   
3      Married-civ-spouse     Other-service        Husband  White    Male   
4      Married-civ-spouse                 ?           Wife  White  Female   
...                   ...               ...            ...    ...     ...   
24995       Never-married      Adm-clerical  Not-in-family  White  Female   
24996       Never-married    Prof-specialty  Not-in-family  White  Female   
24997  Married-civ-spouse    Prof-specialty        Husband  White    Male   
24998       Never-married    Prof-specialty  Not-in-family  White  Female   
24999  Married-civ-spouse  Transport-moving        Husband  White    Male   

       capital.gain  capital.loss  hours.per.week native.country income  
0             99999             0              70  United-States   >50K  
1                 0             0              40  United-States  <=50K  
2                 0             0              40  United-States   >50K  
3                 0             0              60  United-States   >50K  
4                 0             0              40         Mexico  <=50K  
...             ...           ...             ...            ...    ...  
24995             0             0              40  United-States  <=50K  
24996             0             0              50  United-States  <=50K  
24997             0             0              40              ?   >50K  
24998             0             0              40  United-States  <=50K  
24999             0             0              40  United-States  <=50K  

[25000 rows x 15 columns]

In [309]:
df=df.drop('education.num',axis=1)
df=df.drop('fnlwgt',axis=1)


In [310]:
df.replace('?', np.nan, inplace=True)


In [311]:
df.isnull().sum()

age                  0
workclass         1429
education            0
marital.status       0
occupation        1434
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     437
income               0
dtype: int64

In [312]:
#ako imaavme missing values za kategoriski
categorical_columns = ['workclass','occupation','native.country']
imputer = SimpleImputer(strategy = 'most_frequent')
df[categorical_columns] = imputer.fit_transform(df[categorical_columns])
df

age         workclass     education      marital.status  \
0       40  Self-emp-not-inc   Prof-school  Married-civ-spouse   
1       30           Private       HS-grad            Divorced   
2       46           Private  Some-college  Married-civ-spouse   
3       32           Private     Assoc-voc  Married-civ-spouse   
4       54           Private     Preschool  Married-civ-spouse   
...    ...               ...           ...                 ...   
24995   40           Private  Some-college       Never-married   
24996   31         Local-gov     Bachelors       Never-married   
24997   38       Federal-gov     Bachelors  Married-civ-spouse   
24998   23           Private     Bachelors       Never-married   
24999   60           Private          10th  Married-civ-spouse   

             occupation   relationship   race     sex  capital.gain  \
0        Prof-specialty        Husband  White    Male         99999   
1          Craft-repair  Not-in-family  White  Female             0   
2          Adm-clerical        Husband  White    Male             0   
3         Other-service        Husband  White    Male             0   
4        Prof-specialty           Wife  White  Female             0   
...                 ...            ...    ...     ...           ...   
24995      Adm-clerical  Not-in-family  White  Female             0   
24996    Prof-specialty  Not-in-family  White  Female             0   
24997    Prof-specialty        Husband  White    Male             0   
24998    Prof-specialty  Not-in-family  White  Female             0   
24999  Transport-moving        Husband  White    Male             0   

       capital.loss  hours.per.week native.country income  
0                 0              70  United-States   >50K  
1                 0              40  United-States  <=50K  
2                 0              40  United-States   >50K  
3                 0              60  United-States   >50K  
4                 0              40         Mexico  <=50K  
...             ...             ...            ...    ...  
24995             0              40  United-States  <=50K  
24996             0              50  United-States  <=50K  
24997             0              40  United-States   >50K  
24998             0              40  United-States  <=50K  
24999             0              40  United-States  <=50K  

[25000 rows x 13 columns]

In [313]:
df.isnull().sum()

age               0
workclass         0
education         0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

In [314]:
low_income_percentage = (df['income'] == '<=50K').mean() * 100
print(f'Percentage of people earning <=50K: {low_income_percentage}%')

Percentage of people earning <=50K: 75.82%


In [315]:
df

age         workclass     education      marital.status  \
0       40  Self-emp-not-inc   Prof-school  Married-civ-spouse   
1       30           Private       HS-grad            Divorced   
2       46           Private  Some-college  Married-civ-spouse   
3       32           Private     Assoc-voc  Married-civ-spouse   
4       54           Private     Preschool  Married-civ-spouse   
...    ...               ...           ...                 ...   
24995   40           Private  Some-college       Never-married   
24996   31         Local-gov     Bachelors       Never-married   
24997   38       Federal-gov     Bachelors  Married-civ-spouse   
24998   23           Private     Bachelors       Never-married   
24999   60           Private          10th  Married-civ-spouse   

             occupation   relationship   race     sex  capital.gain  \
0        Prof-specialty        Husband  White    Male         99999   
1          Craft-repair  Not-in-family  White  Female             0   
2          Adm-clerical        Husband  White    Male             0   
3         Other-service        Husband  White    Male             0   
4        Prof-specialty           Wife  White  Female             0   
...                 ...            ...    ...     ...           ...   
24995      Adm-clerical  Not-in-family  White  Female             0   
24996    Prof-specialty  Not-in-family  White  Female             0   
24997    Prof-specialty        Husband  White    Male             0   
24998    Prof-specialty  Not-in-family  White  Female             0   
24999  Transport-moving        Husband  White    Male             0   

       capital.loss  hours.per.week native.country income  
0                 0              70  United-States   >50K  
1                 0              40  United-States  <=50K  
2                 0              40  United-States   >50K  
3                 0              60  United-States   >50K  
4                 0              40         Mexico  <=50K  
...             ...             ...            ...    ...  
24995             0              40  United-States  <=50K  
24996             0              50  United-States  <=50K  
24997             0              40  United-States   >50K  
24998             0              40  United-States  <=50K  
24999             0              40  United-States  <=50K  

[25000 rows x 13 columns]

In [316]:
categorical= ['workclass','education','marital.status','occupation','relationship','race','sex','native.country','income']

In [317]:
label_encoder=LabelEncoder()

In [318]:
for col in categorical:
    df[col]= label_encoder.fit_transform(df[col])

In [319]:
df

age  workclass  education  marital.status  occupation  relationship  \
0       40          5         14               2           9             0   
1       30          3         11               0           2             1   
2       46          3         15               2           0             0   
3       32          3          8               2           7             0   
4       54          3         13               2           9             5   
...    ...        ...        ...             ...         ...           ...   
24995   40          3         15               4           0             1   
24996   31          1          9               4           9             1   
24997   38          0          9               2           9             0   
24998   23          3          9               4           9             1   
24999   60          3          0               2          13             0   

       race  sex  capital.gain  capital.loss  hours.per.week  native.country  \
0         4    1         99999             0              70              37   
1         4    0             0             0              40              37   
2         4    1             0             0              40              37   
3         4    1             0             0              60              37   
4         4    0             0             0              40              24   
...     ...  ...           ...           ...             ...             ...   
24995     4    0             0             0              40              37   
24996     4    0             0             0              50              37   
24997     4    1             0             0              40              37   
24998     4    0             0             0              40              37   
24999     4    1             0             0              40              37   

       income  
0           1  
1           0  
2           1  
3           1  
4           0  
...       ...  
24995       0  
24996       0  
24997       1  
24998       0  
24999       0  

[25000 rows x 13 columns]

In [320]:
cluster=['age','workclass','education','marital.status','occupation','relationship','race','sex','income','hours.per.week']

In [321]:
X = df[cluster]

In [322]:
list =[]
for i in range (1,11):
    kmeans = KMeans(n_clusters=i,random_state=42)
    kmeans.fit(X)
    list.append(kmeans.inertia_)

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(range(1, 11), list,marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distance')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.show()

In [ ]:
kmeans1=KMeans(n_clusters=3,random_state=42)
labels = kmeans1.fit_predict(X)

In [ ]:
pca=PCA(n_components=2)
X_pca=pca.fit_transform(X)

In [ ]:
df['cluster'] = labels

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=df['cluster'], palette='viridis',legend='full')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('2D PCA Visualization of Clusters')
plt.show()